<a href="https://colab.research.google.com/github/audwls4545/metaverse/blob/main/pytorch_boston1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import fetch_openml

dataset = fetch_openml(name = 'boston')

dataset.keys()

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:301: UserWarning: Multiple active versions of the dataset matching the name boston exist. Versions may be fundamentally different, returning version 1.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [2]:
import pandas as pd

df = pd.DataFrame(dataset['data'])
df.columns = dataset['feature_names']
df['target'] = dataset['target']

df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [12]:
df['CHAS'] = df['CHAS'].astype('float')
df['RAD'] = df['RAD'].astype('float')

In [13]:
import torch
import torch.nn as nn
from torch.optim.adam import Adam

In [14]:
model = nn.Sequential(
    nn.Linear(13, 100), #13입력, 100출력
    nn.ReLU(),
    nn.Linear(100, 1)

)

model

Sequential(
  (0): Linear(in_features=13, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=1, bias=True)
)

In [15]:
X_train = df.iloc[:,:13].values
X_train.shape

(506, 13)

In [16]:
y_train = df.iloc[:,-1].values
y_train.shape

(506,)

In [17]:
batch_size = 100
optim = Adam(model.parameters(), lr = 0.001)
epochs = 200

for epoch in range(epochs):
    for temp in range(len(X_train) // batch_size):
        start = temp * batch_size
        end = start + batch_size

        x = torch.FloatTensor(X_train[start : end])
        y = torch.FloatTensor(y_train[start : end])

        optim.zero_grad()
        pred = model(x)
        loss = nn.MSELoss()(pred, y)
        loss.backward()
        optim.step()

    if epoch % 10 == 0:
        print(f'{epoch+1} loss:{loss.item()}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


1 loss:71.03168487548828
11 loss:86.46977233886719
21 loss:43.10759353637695
31 loss:38.869815826416016
41 loss:38.05921173095703
51 loss:37.70018005371094
61 loss:37.55073928833008
71 loss:37.395694732666016
81 loss:37.3018798828125
91 loss:37.19560241699219
101 loss:37.098838806152344
111 loss:37.05624008178711
121 loss:37.038570404052734
131 loss:37.01359176635742
141 loss:37.01557922363281
151 loss:37.008270263671875
161 loss:36.988216400146484
171 loss:36.96589660644531
181 loss:36.94986343383789
191 loss:36.93323516845703


In [18]:
X_train[0,:13]

array([6.320e-03, 1.800e+01, 2.310e+00, 0.000e+00, 5.380e-01, 6.575e+00,
       6.520e+01, 4.090e+00, 1.000e+00, 2.960e+02, 1.530e+01, 3.969e+02,
       4.980e+00])

In [19]:
pred = model(torch.FloatTensor(X_train[0,:13]))
pred

tensor([25.3984], grad_fn=<AddBackward0>)

In [20]:
y_train[0]

24.0

In [21]:
model.state_dict()

OrderedDict([('0.weight',
              tensor([[ 0.1412,  0.2062,  0.2333,  ..., -0.0182, -0.1346, -0.2661],
                      [-0.2729, -0.2454, -0.2908,  ...,  0.1069,  0.1833,  0.1192],
                      [ 0.2831,  0.2252,  0.2838,  ...,  0.1578,  0.1096,  0.2744],
                      ...,
                      [-0.2033,  0.1679,  0.1938,  ..., -0.1086, -0.1421,  0.2134],
                      [ 0.1389, -0.0719, -0.0154,  ..., -0.2546, -0.2011,  0.1315],
                      [ 0.1536, -0.0551,  0.2720,  ..., -0.1419, -0.0685,  0.1104]])),
             ('0.bias',
              tensor([-0.0173, -0.1946,  0.2168, -0.0454, -0.1897,  0.1904, -0.2373, -0.2632,
                       0.0664, -0.1678,  0.0920,  0.1388, -0.1767,  0.2509,  0.0518, -0.0115,
                       0.0090,  0.0390,  0.1467, -0.0028, -0.1731, -0.1713,  0.2163,  0.1465,
                       0.0097, -0.1411, -0.1565, -0.0310, -0.0732,  0.0069, -0.1907,  0.0887,
                       0.2378,  0.0332, 

In [22]:
torch.save(model.state_dict(), 'model.pth') #저장

In [ ]:
model_new = model()
model_new.load_state_dict(torch.load('model.pth')) #저장한것 불러오기